In [15]:
import time
import pandas as pd
from difflib import SequenceMatcher
from llama_cpp import Llama

In [20]:
MODEL_PATH = "./models/EXAONE-3.5-2.4B-Instruct-Q4_K_M.gguf"

TEST_DATASET = [
    {"input": "카드 갑씨 왜 이렇게 많이 나왔어요", "truth": "카드 값이 왜 이렇게 많이 나왔어요"},
    {"input": "리벌빙 신청 항거 취소할려구요", "truth": "리볼빙 신청한 거 취소하려고요"},
    {"input": "이번딸 걀제일이 언제여", "truth": "이번 달 결제일이 언제예요"},
    {"input": "한두 상양 좀 부탁드려여", "truth": "한도 상향 좀 부탁드려요"},
    {"input": "일씨불로 긁었는데 할부로 바꿀수 이써요?", "truth": "일시불로 긁었는데 할부로 바꿀 수 있어요?"},
    {"input": "채크카드 재발급 바들려구요", "truth": "체크카드 재발급 받으려고요"},
    {"input": "아니 그 머냐 결제 취소가 안대자나요", "truth": "결제 취소가 안 되잖아요"},
    {"input": "해외겨제 차단 푸러주세요", "truth": "해외결제 차단 풀어주세요"}
]

SYSTEM_PROMPT = """
당신은 금융권 회사의 맞춤법 교정 전문가입니다.
사용자가 입력하는 문장은 발음 오류, 오타, 띄어쓰기 오류가 포함된 텍스트입니다.

[지시사항]
0. 입력된 질문에 대해 답변하지 말고, 해당 문장의 문법을 교정하십시오.
1. 입력된 문장의 의미를 파악하여 정확한 문법과 카드사 전문 용어로 교정하십시오.
2. 불필요한 추임새(음, 그, 머냐 등)는 제거하십시오.
3. 숫자는 가급적 아라비아 숫자(예: 3개월)로 표기하십시오.
4. 절대 부가적인 설명이나 인사를 하지 마십시오. 오직 '교정된 텍스트'만 출력하십시오.

[예시]
입력: 카드 갑씨 왜 이렇게 많이 나왔어요
교정: 카드 값이 왜 이렇게 많이 나왔어요

입력: 리벌빙 신청 항거 취소할려구요
교정: 리볼빙 신청한 거 취소하려고요
"""

In [ ]:
class STTFilterEvaluator:
    def __init__(self, model_path):
        print(f"🔄 Loading Correction Model...")
        try:
            self.llm = Llama(
                model_path=model_path,
                n_gpu_layers=-1,
                n_ctx=2048,
                verbose=False
            )
            print("✅ Model Loaded.")
        except Exception as e:
            print(f"❌ Load Failed: {e}")
            self.llm = None

    def correct_text(self, noisy_text):
        if not self.llm: return "Error"
        
        try:
            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"입력: {noisy_text}"}
            ]
            
            output = self.llm.create_chat_completion(
                messages=messages,
                max_tokens=128,     # 교정 텍스트는 길지 않으므로 짧게 제한
                temperature=0.1     # 창의성 억제 (정확도 우선)
            )
            return output['choices'][0]['message']['content'].strip()
        except ValueError:
             # System role 미지원 모델(Gemma 등) Fallback
            prompt = f"{SYSTEM_PROMPT}\n\n입력: {noisy_text}\n교정 결과:"
            output = self.llm.create_completion(
                prompt=prompt, max_tokens=128, temperature=0.1
            )
            return output['choices'][0]['text'].strip()

    def calculate_similarity(self, a, b):
        return SequenceMatcher(None, a, b).ratio()

In [ ]:
def main():
    evaluator = STTFilterEvaluator(MODEL_PATH)
    if not evaluator.llm: return

    results = []

    for idx, data in enumerate(TEST_DATASET):
        noisy_input = data['input']
        ground_truth = data['truth']

        start_time = time.time()
        corrected_output = evaluator.correct_text(noisy_input)
        elapsed = time.time() - start_time

        # 유사도 점수 (100점 만점)
        score = evaluator.calculate_similarity(ground_truth, corrected_output) * 100

        print(f"Input: {noisy_input}")
        print(f"Pred : {corrected_output}")
        print(f"Truth: {ground_truth}")
        print(f"Score: {score:.1f} / Time: {elapsed:.2f}s\n")

        results.append({
            "Noisy Input": noisy_input,
            "Model Output": corrected_output,
            "Ground Truth": ground_truth,
            "Similarity": round(score, 1),
            "Time(s)": round(elapsed, 3)
        })

    # 결과 저장
    df = pd.DataFrame(results)

In [21]:
if __name__ == "__main__":
    main()

🔄 Loading Correction Model...


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


✅ Model Loaded.

🚀 [STT Correction Test Started]

Input: 카드 갑씨 왜 이렇게 많이 나왔어요
Pred : 교정: 카드 이용금액이 왜 이렇게 많이 나왔나요?
Truth: 카드 값이 왜 이렇게 많이 나왔어요
Score: 73.9 / Time: 2.99s

Input: 리벌빙 신청 항거 취소할려구요
Pred : 리볼빙 신청을 취소하려고 합니다.
Truth: 리볼빙 신청한 거 취소하려고요
Score: 70.6 / Time: 0.72s

Input: 이번딸 걀제일이 언제여
Pred : 교정: 이번 체크일이 언제인가요?
Truth: 이번 달 결제일이 언제예요
Score: 56.2 / Time: 0.74s

Input: 한두 상양 좀 부탁드려여
Pred : 입력: 한두 상량 좀 부탁드리겠습니다.
Truth: 한도 상향 좀 부탁드려요
Score: 52.9 / Time: 0.77s

Input: 일씨불로 긁었는데 할부로 바꿀수 이써요?
Pred : 입력: 일씨불로 긁었는데 할부로 바꿀 수 있나요?
Truth: 일시불로 긁었는데 할부로 바꿀 수 있어요?
Score: 84.0 / Time: 1.09s

Input: 채크카드 재발급 바들려구요
Pred : 교정: 체크카드 재발급 요청드립니다.
Truth: 체크카드 재발급 받으려고요
Score: 58.8 / Time: 0.72s

Input: 아니 그 머냐 결제 취소가 안대자나요
Pred : 아니요, 결제 취소가 불가능하지 않나요?
Truth: 결제 취소가 안 되잖아요
Score: 51.4 / Time: 0.88s

Input: 해외겨제 차단 푸러주세요
Pred : 입력: 해외거래 차단 해제 부탁드립니다.
Truth: 해외결제 차단 풀어주세요
Score: 34.3 / Time: 0.62s

